In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
import torch, platform, sys
print("Python:", sys.version.split()[0], "| Platform:", platform.machine())
print("PyTorch:", torch.__version__)
print("CUDA:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

Python: 3.12.11 | Platform: x86_64
PyTorch: 2.8.0+cu126
CUDA: False


In [ ]:
!pip -q install -U transformers datasets evaluate rouge-score bert-score sentencepiece accelerate tiktoken einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 105.3 MB/s eta 0:00:00


In [ ]:
!pip -q install langdetect ftfy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 26.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.5 MB/s eta 0:00:00


In [ ]:
!mkdir -p moodmuse/{data,models,src}

In [ ]:
from pathlib import Path

ROOT = Path("/content/MoodMuse")
for p in [
    ROOT/"data/raw",
    ROOT/"data/processed",
    ROOT/"outputs/checkpoints",
    ROOT/"outputs/samples",
    ROOT/"src",
]:
    p.mkdir(parents=True, exist_ok=True)

# quick config helper we'll reuse
CONFIG = {
    "root": str(ROOT),
    "raw_dir": str(ROOT/"data/raw"),
    "proc_dir": str(ROOT/"data/processed"),
    "ckpt_dir": str(ROOT/"outputs/checkpoints"),
    "samples_dir": str(ROOT/"outputs/samples"),
}
CONFIG

{'root': '/content/MoodMuse',
 'raw_dir': '/content/MoodMuse/data/raw',
 'proc_dir': '/content/MoodMuse/data/processed',
 'ckpt_dir': '/content/MoodMuse/outputs/checkpoints',
 'samples_dir': '/content/MoodMuse/outputs/samples'}

In [ ]:
import os, json, pathlib
from google.colab import files

kaggle_dir = pathlib.Path("/root/.kaggle")
kaggle_dir.mkdir(parents=True, exist_ok=True)

if not (kaggle_dir/"kaggle.json").exists():
    print("⬆️ Upload kaggle.json")
    uploaded = files.upload()
    with open(kaggle_dir/"kaggle.json", "wb") as f:
        f.write(uploaded["kaggle.json"])
    os.chmod(kaggle_dir/"kaggle.json", 0o600)

!kaggle --version

⬆️ Upload kaggle.json


Saving kaggle.json to kaggle.json
Kaggle API 1.7.4.5


In [ ]:
!kaggle --version
!ls -l /root/.kaggle/kaggle.json

Kaggle API 1.7.4.5
-rw------- 1 root root 69 Sep  2 16:00 /root/.kaggle/kaggle.json


In [ ]:
!mkdir -p data

In [ ]:
!kaggle datasets list -s "instagram caption" -v
!kaggle datasets list -s "social media captions" -v

ref,title,size,lastUpdated,downloadCount,voteCount,usabilityRating
prithvijaunjale/instagram-images-with-captions,Instagram Images with Captions,4120862937,2020-03-21 11:34:12.083000,2772,59,0.4117647
propriyam/instagram-data,Instagram Data,3349500,2024-03-14 06:41:29.870000,1908,33,1.0
thecoderenroute/instagram-posts-dataset,Instagram posts dataset,1815072617,2023-05-09 11:51:56.433000,1251,18,0.875
rxsraghavagrawal/instagram-reach,Instagram Reach,17260,2021-01-21 13:34:20.880000,2585,15,0.47058824
faiznfi/surabaa-high-school-instagram-caption,Surabaya High School Instagram Caption,137314,2018-08-13 02:28:28.227000,83,2,0.5882353
amankumarkharwal/instagram-reach-data,Instagram Reach Data,11201,2022-04-07 17:04:25.450000,258,6,0.47058824
krpurba/fakeauthentic-user-instagram,Fake/Authentic User Instagram,3495878,2023-11-05 01:06:16.493000,1034,19,0.7058824
rezaunderfit/what-people-are-talking-about-pfizer-on-instagram,Pfizer Instagram posts,250793,2021-01-19 07:03:51.647000,106,5,0.7647

In [ ]:
DATASET = "propriyam/instagram-data"
!kaggle datasets download -d $DATASET -p data --unzip
!ls -R data

Dataset URL: https://www.kaggle.com/datasets/propriyam/instagram-data
License(s): MIT
  0% 0.00/3.19M [00:00<?, ?B/s]
100% 3.19M/3.19M [00:00<00:00, 852MB/s]
data:
instagram_data.csv


Loading and peeking into the dataset

In [ ]:
import pandas as pd

path = "data/instagram_data.csv"
df = pd.read_csv(path, low_memory=False)

df.shape, df.columns.tolist(), df.head(3)

((11692, 14),
 ['owner_id',
  'owner_username',
  'shortcode',
  'is_video',
  'caption',
  'comments',
  'likes',
  'created_at',
  'location',
  'imageUrl',
  'multiple_images',
  'username',
  'followers',
  'following'],
    owner_id     owner_username    shortcode is_video  \
 0  36063641  christendominique  C3_GS1ASeWI    False   
 1  36063641  christendominique  C38ivgNS3IX     True   
 2  36063641  christendominique  C35-Dd9SO1b     True   
 
                                              caption comments  likes  \
 0  I’m a brunch & Iced Coffee girlie☕️🍳 \n\nTop @...      268  16382   
 1  😮‍💨Brow tips I really wish I would have know w...      138   9267   
 2  OMG I can’t believe it’s already been 1 yr sin...     1089  10100   
 
      created_at location                                           imageUrl  \
 0  1.709327e+09      NaN  https://instagram.flba2-1.fna.fbcdn.net/v/t39....   
 1  1.709241e+09      NaN  https://instagram.flba2-1.fna.fbcdn.net/v/t51....   
 2  1.70915

In [ ]:
# basic caption diagnostics
cap_col_candidates = [c for c in df.columns if 'caption' in c.lower() or 'text' in c.lower()]
cap_col_candidates

['caption']

In [ ]:
cap_col = cap_col_candidates[0]  # picking the first match for now
df[cap_col].isna().mean(), df[cap_col].duplicated().mean(), df[cap_col].str.len().describe()

(np.float64(0.013770099213137188),
 np.float64(0.31388983920629493),
 count    11531.000000
 mean       311.179256
 std        396.987874
 min          1.000000
 25%         55.000000
 50%        155.000000
 75%        402.000000
 max       2200.000000
 Name: caption, dtype: float64)

In [ ]:
df[cap_col].dropna().sample(10, random_state=42).tolist()

['Akhirnya buka juga @tokokopituku 😭🙌\n.\n#tokokopituku #eskopibandung #makanpakereceh #bandungfoodies #kulinerbandung #wisatabandung',
 'Creamy tomato burrata pasta 🍅\n\nThis pasta dish is such a great, simple meal for the Summertime! It’s super easy to make and absolutely delicious, hope you enjoy 😋\n\n(Serves two)\n\nIngredients:\u2028- 2 tbsp butter\u2028- 1 garlic clove, minced\u2028- 1 whole chilli, diced\u2028- 250g cherry tomatoes\u2028- 2 tbsp tomato paste\u2028- 1 cup thickened cream\u2028- 250g pasta, cooked al dente\u2028- 150g burrata\u2028- fresh basil leaves, to serve\u2028- salt and pepper, to taste\n\nMethod:\u20281. Cook your pasta and prepare garlic and chilli\u20282. Place a large frying pan over medium-high heat on the stove, add butter and melt\u20283. Add garlic, chilli and cherry tomatoes and fry for 3-4 minutes\u20284. Add tomato paste, salt and pepper and continue frying for a further 3 minutes\u20285. Add thickened cream and reduce heat to low-medium. Continu

Cleaning the Data

In [ ]:
captions = df["caption"].dropna().drop_duplicates().str.strip()

# Removing very short captions (the ones maybe less 5 chars)

captions = captions[captions.str.len() >= 5]

# Restting the index

captions = captions.reset_index(drop=True)

print("Final size:", len(captions))
captions.sample(10, random_state=42).tolist()

Final size: 7906


['Swimwear that matches the ocean 💙 @monycaeleogram wearing our most supportive full piece Rhea in Lapis Terry Ribbed 🌞 \nVideo by @sierrasavannahf \n.\n#super8 #swimwear #super8film #swimwearfashion #hawaiistagram',
 'Desliza para saltar! Rétanos acá abajo estaremos cumpliendo los que tengan más likes',
 'Paris dump 📸\n難得在時裝周的一日假期\n雖然也用了半天的時間拍攝工作\n但今早起床也有叉過電的感覺💆🏻\u200d♀️\n下午就是期待已久的valentino show～時尚快遞很快繼續更新🤘🏻\n#elvainpfw\n#orsaymuseum',
 'new beginnings 🏋🏼\u200d♀️ @crossfitlevanger \n#crossfitlevanger #crossfit #levanger #sprek',
 'Keeping it classy for the @hermes show',
 'Anzeige // thank you for having me @rabanne 🤩 (PSA: I did not wake up like this) #rabanneparfums',
 'Walks around #Wapping 🌚',
 'mountain ⛷️ time',
 'New York’a iner inmez ayağımın tozuyla  @burak.piri.araz ‘ı alıp doğru Central Park’ın yolunu tuttum. Metrodan Time Square’de inip başladık yürümeye ilk durağımız tabii ki İtalyan pizzacısı kaptık ikişer dilim pizzayı🍕🍕yola devam…\n\nHava mı?Hava buzzz 1-2 derecelerde.

In [ ]:
out_path = "data/captions_clean.txt"

with open(out_path, "w", encoding="utf-8") as f:
    for cap in captions:
        f.write(cap + "\n")

print(f"Saved {len(captions)} captions to {out_path}")

Saved 7906 captions to data/captions_clean.txt


Filtering cleaned captions to English

In [ ]:
from langdetect import detect_langs, DetectorFactory
from pathlib import Path
import re
from ftfy import fix_text

DetectorFactory.seed = 0  # make langdetect deterministic

src_path = Path("data/captions_clean.txt")
lines = [ln.strip() for ln in src_path.read_text(encoding="utf-8").splitlines()]
lines = [ln for ln in lines if ln]  # drop empties

url_pat = re.compile(r"https?://\S+|www\.\S+")
handle_pat = re.compile(r"@\w+")
hash_pat = re.compile(r"#\w+")
emoji_pat = re.compile(
    "[\U0001F300-\U0001F6FF\U0001F900-\U0001F9FF\U00002700-\U000027BF\U00002600-\U000026FF]"
)

def ascii_ratio(s: str) -> float:
    if not s: return 0.0
    a = sum(1 for ch in s if ord(ch) < 128)
    return a / len(s)

def is_english(s: str) -> bool:
    # light cleaning for detection only (keeping original text)
    probe = url_pat.sub(" ", s)
    probe = handle_pat.sub(" ", probe)
    probe = hash_pat.sub(" ", probe)
    probe = emoji_pat.sub(" ", probe)
    probe = fix_text(probe).strip()

    # quick heuristic guards (helps the detector)
    if ascii_ratio(probe) < 0.7:
        return False
    if len(probe) < 12:
        # tiny snippets confuse detectors; allow short ASCII-ish lines
        return True

    try:
        langs = detect_langs(probe)
        # keeping if English has decent probability
        for lp in langs:
            if lp.lang == "en" and lp.prob >= 0.80:
                return True
        return False
    except Exception:
        # fallback on heuristic if detector fails
        return ascii_ratio(probe) > 0.85

english_lines = [ln for ln in lines if is_english(ln)]

out_path = Path("data/captions_en.txt")
out_path.write_text("\n".join(english_lines), encoding="utf-8")
print(f"Kept {len(english_lines)} / {len(lines)} captions as English-only → {out_path}")


Kept 16594 / 28622 captions as English-only → data/captions_en.txt


Splitting into train / validation

In [ ]:
from pathlib import Path
import random

random.seed(42)

src = Path("data/captions_en.txt")   # << using the English-only file
lines = [ln.rstrip("\n") for ln in src.read_text(encoding="utf-8").splitlines()]
lines = [ln for ln in lines if ln.strip()]

random.shuffle(lines)

n = len(lines)
n_val = max(1, int(0.10 * n))
val_lines = lines[:n_val]
train_lines = lines[n_val:]

Path("data").mkdir(parents=True, exist_ok=True)
Path("data/train.txt").write_text("\n".join(train_lines), encoding="utf-8")
Path("data/val.txt").write_text("\n".join(val_lines), encoding="utf-8")

print(f"Total EN: {n} | train: {len(train_lines)} | val: {len(val_lines)}")
print("train sample ->", train_lines[0][:120])
print("val   sample ->", val_lines[0][:120])

Total EN: 16594 | train: 14935 | val: 1659
train sample -> Steadicam @distefano
val   sample -> Why did the sugar story not publish?


In [ ]:
import json
from pathlib import Path

train_text = Path("data/train.txt").read_text(encoding="utf-8")

# Building vocabulary from TRAIN only (avoid val leakage)

chars = sorted(list(set(train_text)))
itos = {i:c for i,c in enumerate(chars)}
stoi = {c:i for i,c in itos.items()}

def encode(s: str):
  return [stoi[c] for c in s if c in stoi]

def decode(ids):
  return "".join(itos[i] for i in ids)

print("Vocab size:", len(chars))
print("Sample encode/decode check:")
test = "mood muse"
print(encode(test)[:20], "->", decode(encode(test)))


tok_path = Path("data/tokenizer_char.json")
tok_path.write_text(json.dumps({"stoi": stoi, "itos": itos}, ensure_ascii=False), encoding="utf-8")
print("Saved tokenizer to", tok_path)

Vocab size: 1154
Sample encode/decode check:
[79, 81, 81, 70, 2, 79, 87, 85, 71] -> mood muse
Saved tokenizer to data/tokenizer_char.json


In [ ]:
# token-length distribution on a small slice (for speed)
sampled = train_lines[:1000]
lengths = [len(encode(x+"\n")) for x in sampled]  # include newline like classic text corpora
print(f"Avg tokens: {sum(lengths)/len(lengths):.1f} | max: {max(lengths)} | min: {min(lengths)}")

Avg tokens: 107.4 | max: 1278 | min: 2


In [ ]:
import torch, math
from pathlib import Path
import json

# load tokenizer

tok = json.loads(Path("data/tokenizer_char.json").read_text(encoding="utf-8"))

# Ensure stoi keys are strings and values are integers
stoi = {k: int(v) for k, v in tok["stoi"].items()}

# Ensure itos keys are integers and values are strings
itos = {int(k): v for k, v in tok["itos"].items()}


def encode(s): return torch.tensor([stoi[c] for c in s if c in stoi], dtype=torch.long)
def decode(ids): return "".join(itos[int(i)] for i in ids) # Added .item() to extract scalar from tensor


# Read data

train_text = Path("data/train.txt").read_text(encoding="utf-8")
val_text = Path("data/val.txt").read_text(encoding="utf-8")

# Simple separator to help the model see caption boundaries

SEP = "\n"
train_ids = encode(train_text + SEP)
val_ids = encode(val_text + SEP)

vocab_size = len(itos)
print("train tokens:", len(train_ids), " | val tokens:", len(val_ids), " | vocab:", vocab_size)

# Batching utils

device = "cuda" if torch.cuda.is_available() else "cpu"
block_size = 128 # Corrected black_size to block_size based on usage below
batch_size = 64

def get_batch(split): # Corrected splits to split based on usage below
  data = train_ids if split == "train" else val_ids
  ix = torch.randint(0, len(data) - block_size - 1, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+1+block_size] for i in ix])
  return x.to(device), y.to(device)

train tokens: 1599397  | val tokens: 167511  | vocab: 1154


Defining the model and running a short training loop

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(42)

n_embd   = 256
n_head   = 4
n_layer  = 4
dropout  = 0.2
lr       = 3e-4

class CausalSelfAttention(nn.Module):
    def __init__(self, n_embd, n_head, dropout):
        super().__init__()
        assert n_embd % n_head == 0
        self.n_head = n_head
        self.head_dim = n_embd // n_head
        self.qkv = nn.Linear(n_embd, 3*n_embd, bias=False)
        self.proj = nn.Linear(n_embd, n_embd, bias=False)
        self.attn_drop = nn.Dropout(dropout)
        self.resid_drop = nn.Dropout(dropout)
        self.register_buffer("mask", torch.tril(torch.ones(block_size, block_size)).unsqueeze(0).unsqueeze(0))

    def forward(self, x):
        B,T,C = x.size()
        q,k,v = self.qkv(x).chunk(3, dim=-1)
        q = q.view(B,T,self.n_head,self.head_dim).transpose(1,2)
        k = k.view(B,T,self.n_head,self.head_dim).transpose(1,2)
        v = v.view(B,T,self.n_head,self.head_dim).transpose(1,2)
        att = (q @ k.transpose(-2,-1)) / (self.head_dim ** 0.5)
        att = att.masked_fill(self.mask[:,:,:T,:T]==0, float("-inf"))
        att = F.softmax(att, dim=-1)
        att = self.attn_drop(att)
        y = att @ v
        y = y.transpose(1,2).contiguous().view(B,T,C)
        y = self.resid_drop(self.proj(y))
        return y

class MLP(nn.Module):
    def __init__(self, n_embd, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.GELU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout),
        )
    def forward(self, x): return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head, dropout):
        super().__init__()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        self.attn = CausalSelfAttention(n_embd, n_head, dropout)
        self.mlp  = MLP(n_embd, dropout)
    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

class TinyGPT(nn.Module):
    def __init__(self, vocab_size, n_embd, n_head, n_layer, dropout):
        super().__init__()
        self.tok_emb = nn.Embedding(vocab_size, n_embd)
        self.pos_emb = nn.Embedding(block_size, n_embd)
        self.drop = nn.Dropout(dropout)
        self.blocks = nn.ModuleList([Block(n_embd, n_head, dropout) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size, bias=False)

        self.apply(self._init)
    def _init(self, m):
        if isinstance(m, (nn.Linear, nn.Embedding)):
            nn.init.normal_(m.weight, mean=0.0, std=0.02)
        if isinstance(m, nn.Linear) and m.bias is not None:
            nn.init.zeros_(m.bias)

    @torch.no_grad()
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            probs = F.softmax(logits[:, -1, :], dim=-1)
            next_id = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, next_id], dim=1)
        return idx

    def forward(self, idx, targets=None):
        B,T = idx.shape
        tok = self.tok_emb(idx)
        pos = self.pos_emb(torch.arange(T, device=idx.device))
        x = self.drop(tok + pos)
        for blk in self.blocks:
            x = blk(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(B*T, -1), targets.view(B*T))
        return logits, loss

model = TinyGPT(vocab_size, n_embd, n_head, n_layer, dropout).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, betas=(0.9, 0.95), weight_decay=0.01)

max_steps = 600
log_every = 100

model.train()
for step in range(1, max_steps+1):
    xb, yb = get_batch("train")
    optimizer.zero_grad(set_to_none=True)
    _, loss = model(xb, yb)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    if step % log_every == 0:
        with torch.no_grad():
            xbv, ybv = get_batch("val")
            _, vloss = model(xbv, ybv)
        print(f"step {step:4d} | train {loss.item():.3f} | val {vloss.item():.3f}")

step  100 | train 2.735 | val 2.763
step  200 | train 2.658 | val 2.679
step  300 | train 2.610 | val 2.659
step  400 | train 2.535 | val 2.580
step  500 | train 2.456 | val 2.511
step  600 | train 2.352 | val 2.378


In [ ]:
prompt = "STYLE: dreamy minimal | SCENE: golden hour at the beach | MOOD: soft & warm | CAPTION:"

In [ ]:
import torch.nn.functional as F

@torch.no_grad()
def sample_text(prompt="", max_new_tokens=120, temperature=0.8, top_k=None, top_p=0.9,
                rep_penalty=1.12, n_samples=5):
    model.eval()
    start = encode(prompt if prompt else "\n").unsqueeze(0).to(device)
    out = []
    for _ in range(n_samples):
        idx = start.clone()
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = model(idx_cond)
            logits = logits[:, -1, :] / max(temperature, 1e-8)

            # repetition penalty (simple token-level)
            if rep_penalty != 1.0:
                seen = torch.bincount(idx.view(-1), minlength=vocab_size).clamp(max=1).float()
                logits = logits - (rep_penalty - 1.0) * seen.unsqueeze(0).to(logits.device)

            # top-k
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float("inf")

            # nucleus (top-p)
            if top_p is not None:
                sorted_logits, sorted_idx = torch.sort(logits, descending=True)
                cumprobs = torch.cumsum(torch.softmax(sorted_logits, dim=-1), dim=-1)
                mask = cumprobs > top_p
                mask[..., 1:] = mask[..., :-1].clone()
                mask[..., 0] = False
                sorted_logits[mask] = -float("inf")
                logits = torch.zeros_like(logits).scatter_(1, sorted_idx, sorted_logits)

            probs = torch.softmax(logits, dim=-1)
            next_id = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, next_id], dim=1)
            if int(next_id) == stoi.get("\n", -999):
                break
        out.append(decode(idx[0].tolist()))
    return out

In [ ]:
import re

def polish(s: str):
    s = re.sub(r'[ \t]+', ' ', s).strip()
    s = re.sub(r'([.!?])\1{2,}', r'\1\1', s)
    s = re.sub(r'([#@]\S+)(?:\s+\1)+', r'\1', s)  # dedupe tags/mentions
    tags = re.findall(r'#\w+', s)
    if len(tags) > 3:
        keep = tags[:3]
        s = re.sub(r'#\w+', '', s)
        s = s.strip() + " " + " ".join(keep)
    return s.strip()


In [ ]:
import torch.nn.functional as F

@torch.no_grad()
def sample_text(prompt="", n_samples=3, max_new_tokens=120, temperature=0.8, top_k=60):
    model.eval()
    start = encode(prompt if prompt else "\n").unsqueeze(0).to(device)
    out = []
    for _ in range(n_samples):
        idx = start.clone()
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = model(idx_cond)
            logits = logits[:, -1, :] / max(temperature, 1e-8)
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('inf')
            probs = F.softmax(logits, dim=-1)
            next_id = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, next_id], dim=1)
            if int(next_id) == stoi.get("\n", -999):
                break
        out.append(decode(idx[0].tolist()))
    model.train()
    return out

In [ ]:
import re

def decode_ascii(ids):
    # Allowing printable ASCII + newline; mapping everything else to space
    out = []
    for i in ids:
        ch = itos[int(i)]
        o = ord(ch)
        if ch == "\n" or (32 <= o <= 126):
            out.append(ch)
        else:
            out.append(" ")
    txt = "".join(out)
    # collapsing weird spaces
    txt = re.sub(r"\s+", " ", txt).strip()
    return txt

def tidy_caption(s, max_len=140):
    # keeping hashtags/@, punctuation; stripping anything non-ASCII that slipped
    s = s.encode("ascii", "ignore").decode("ascii")
    s = re.sub(r"[^A-Za-z0-9#@&.,!?'\"()\-/:; ]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    # trimming length politely at word boundary
    if len(s) > max_len:
        s = s[:max_len]
        s = re.sub(r"\s+\S*$", "", s).rstrip(" ,;:-")
        s += "..."
    # basic sentence case if looks shouty/garbled
    if s and s == s.upper():
        s = s.capitalize()
    # ensuring closing punctuation
    if s and s[-1] not in ".!?":
        s += "."
    return s

@torch.no_grad()
def sample_text_ascii(prompt="", n_samples=3, max_new_tokens=90, temperature=0.7, top_k=50):
    model.eval()
    start = encode(prompt if prompt else "\n").unsqueeze(0).to(device)
    outs = []
    for _ in range(n_samples):
        idx = start.clone()
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = model(idx_cond)
            logits = logits[:, -1, :] / max(temperature, 1e-8)
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('inf')
            probs = F.softmax(logits, dim=-1)
            next_id = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, next_id], dim=1)
            if int(next_id) == stoi.get("\n", -999):
                break
        raw = decode_ascii(idx[0].tolist())
        outs.append(tidy_caption(raw))
    model.train()
    return outs


In [ ]:
outs = sample_text_ascii(prompt="golden hour by the sea #moody", n_samples=3, temperature=0.75, top_k=60, max_new_tokens=90)
for i, s in enumerate(outs, 1):
    print(f"\n--- sample {i} ---\n{s}")


--- sample 1 ---
golden hour by the sea #moody a g PO X ;Y g p K P K P.

--- sample 2 ---
golden hour by the sea #moody X G O D Y ; P ".

--- sample 3 ---
golden hour by the sea #moodyg e X a C S r P Xn ".


In [ ]:
import re, random, math
import torch.nn.functional as F

# 1) allowing only nice printable chars and tidying up
SAFE_CHARS = set("\n " + "".join([chr(i) for i in range(33,127)]))
ALNUM_PUNCT = re.compile(r"[^A-Za-z0-9#@&.,!?'\"()\-/:; ]+")

def decode_clean(ids):
    out = []
    for i in ids:
        ch = itos[int(i)]
        if ch in SAFE_CHARS:
            out.append(ch)
        else:
            out.append(" ")
    txt = "".join(out)
    txt = re.sub(r"\s+", " ", txt).strip()
    return txt

def tidy_line(s, max_len=140):
    s = s.encode("ascii","ignore").decode("ascii")
    s = ALNUM_PUNCT.sub(" ", s)
    s = re.sub(r"\s+", " ", s).strip()
    if len(s) > max_len:
        s = s[:max_len]
        s = re.sub(r"\s+\S*$", "", s).rstrip(" ,;:-") + "..."
    if s and s[-1] not in ".!?":
        s += "."
    # cheap “de-soup”: removing sequences of isolated single capitals
    s = re.sub(r"(?<=\s)[A-Z](?=\s)", "", s)
    s = re.sub(r"\s{2,}", " ", s).strip()
    return s

# 2) generating one sample with stricter settings
@torch.no_grad()
def sample_once(prompt, max_new_tokens=80, temperature=0.6, top_k=40, stop_on_newline=True):
    model.eval()
    idx = encode((prompt or "\n")).unsqueeze(0).to(device)
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -block_size:]
        logits, _ = model(idx_cond)
        logits = logits[:, -1, :] / max(temperature, 1e-8)
        if top_k is not None:
            v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
            logits[logits < v[:, [-1]]] = -float("inf")
        probs = F.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, num_samples=1)
        idx = torch.cat([idx, next_id], dim=1)
        if stop_on_newline and int(next_id) == stoi.get("\n", -999):
            break
    raw = decode_clean(idx[0].tolist())
    return tidy_line(raw)

# 3) scoring caption for “English-ness”
VOWELS = set("aeiouAEIOU")
def score_caption(s):
    if not s: return -1e9
    L = len(s)
    words = s.split()
    # basics
    score = 0.0
    if 25 <= L <= 150: score += 1.0
    score += min(len(words)/12, 1.0)            # more words up to ~12
    # letter fraction
    letters = sum(c.isalpha() for c in s)
    score += (letters / max(1,L)) * 2.0
    # vowel ratio encourages pronounceable text
    vowels = sum(c in VOWELS for c in s)
    if letters > 0: score += (vowels / letters)
    # penalize many digits / weird repeats
    digits = sum(c.isdigit() for c in s)
    score -= 0.5 * (digits / max(1,L))
    if re.search(r"(.)\1\1\1", s): score -= 1.0
    # light bonus for hashtags/mentions but not too many
    tags = len(re.findall(r"(#\w+|@\w+)", s))
    score += min(tags, 2) * 0.2
    if tags > 4: score -= 0.5
    # penalize lots of single-letter tokens
    singles = sum(len(w)==1 for w in words)
    score -= singles * 0.1
    return score

# 4) Best-of-N sampler
def generate_best_caption(desc="", vibe="aesthetic", trials=60):
    style_hint = f" #{vibe.replace(' ','')}" if vibe else ""
    prompt = (desc.strip() + style_hint).strip()
    cands = []
    # small jitter over temperature/top_k to avoid mode collapse
    for _ in range(trials):
        t = random.choice([0.55, 0.6, 0.65, 0.7])
        k = random.choice([30, 40, 50, 60])
        s = sample_once(prompt, temperature=t, top_k=k)
        cands.append((score_caption(s), s))
    cands.sort(reverse=True)
    # return top 3 for safety
    return [s for _, s in cands[:3]]

In [ ]:
import torch, torch.nn.functional as F, re
from collections import Counter

def sample_smart(
    prompt="",
    max_new_tokens=48,         # shorter = punchier
    temperature=0.7,           # calmer wording
    top_k=120,                 # allow more choices
    top_p=0.90,                # nucleus filter
    rep_penalty=1.10,          # nudge away from repetition
    stop_on_punct=True,        # stop after ., !, ?
    n=8,                       # generate many, we'll pick best
    device="cuda" if torch.cuda.is_available() else "cpu",
):
    start = encode((prompt or "\n")).unsqueeze(0).to(device)
    outs = []
    for _ in range(n):
        idx = start.clone()
        seen = Counter()
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = model(idx_cond)
            logits = logits[:, -1, :] / max(temperature, 1e-8)

            # repetition penalty (simple token-level)
            if rep_penalty != 1.0:
                for t, c in seen.items():
                    if c > 0:
                        logits[..., t] /= rep_penalty

            # top-k
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float("inf")
            # top-p (nucleus)
            if top_p is not None:
                sort_logits, sort_idx = torch.sort(logits, descending=True)
                probs = F.softmax(sort_logits, dim=-1)
                cdf = torch.cumsum(probs, dim=-1)
                mask = cdf > top_p
                mask[..., 1:] = mask[..., :-1].clone()
                mask[..., 0] = False
                sort_logits[mask] = -float("inf")
                logits = torch.zeros_like(logits).scatter_(-1, sort_idx, sort_logits)

            probs = F.softmax(logits, dim=-1)
            next_id = torch.multinomial(probs, num_samples=1)
            seen[int(next_id)] += 1
            idx = torch.cat([idx, next_id], dim=1)

            if stop_on_punct:
                ch = itos[int(next_id)]
                if ch in [".", "!", "?"] or ch == "\n":
                    break
        outs.append(decode(idx[0].tolist()))
    return outs


In [ ]:
import re
def tidy_line(s, max_len=140):
    s = s.encode("ascii","ignore").decode("ascii")
    s = re.sub(r"[^A-Za-z0-9#@&.,!?'\"()\-/:; ]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    if len(s) > max_len:
        s = s[:max_len]
        s = re.sub(r"\s+\S*$", "", s).rstrip(" ,;:-")
    return s

ascii_text = "\n".join(tidy_line(ln) for ln in open("data/train.txt", encoding="utf-8"))
open("data/train_ascii.txt", "w", encoding="utf-8").write(ascii_text)
train_ids = encode(ascii_text + "\n").to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, betas=(0.9,0.95), weight_decay=0.01)
for step in range(1, 401):  # 400 quick steps
    ix = torch.randint(0, len(train_ids)-block_size-1, (batch_size,))
    xb = torch.stack([train_ids[i:i+block_size] for i in ix]).to(device)
    yb = torch.stack([train_ids[i+1:i+1+block_size] for i in ix]).to(device)
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True); loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0); optimizer.step()
    if step % 100 == 0: print(f"polish step {step} | loss {loss.item():.3f}")

polish step 100 | loss 2.202
polish step 200 | loss 2.112
polish step 300 | loss 2.086
polish step 400 | loss 2.028


In [ ]:
def clean_caption(s: str):
    s = s.split("\n")[-1]
    s = re.sub(r"[^ -~]+", " ", s)              # ASCII only
    s = re.sub(r"\s+", " ", s).strip()
    # keeping at most 2 hashtags; drop @handles
    parts = s.split()
    kept, tags = [], []
    for w in parts:
        if w.startswith("@"):
            continue
        if w.startswith("#"):
            if len(tags) < 2 and 2 <= len(w) <= 24:
                tags.append(w.lower())
            continue
        kept.append(w)
    core = " ".join(kept).strip(" ,.;:-")
    tail = (" " + " ".join(tags)) if tags else ""
    s = (core + tail).strip()
    # sentence-case & length sweet-spot
    if s and s[0].islower(): s = s[0].upper() + s[1:]
    return s[:140]

def score_caption(s: str):
    # simple heuristic: preferring 6–16 words, no weird repeats
    words = s.split()
    n = len(words)
    length_score = -abs(n - 10)                  # peak at ~10 words
    repeat_pen = -sum(c > 2 for c in Counter(words).values())
    has_emoji_pen = -1 if re.search(r"[^\x20-\x7E]", s) else 0
    hashtag_bonus = min(2, s.count("#")) * 0.5   # a little stylistic bump
    end_punct = 0.5 if s.endswith((".", "!", "?")) else 0
    return length_score + repeat_pen + hashtag_bonus + end_punct + has_emoji_pen

def generate_best(prompt, vibe="moody"):
    raw = sample_smart(f"{prompt} #{vibe}", n=12, max_new_tokens=48, temperature=0.7, top_k=120, top_p=0.9)
    cleaned = [clean_caption(r) for r in raw]
    cleaned = [c for c in cleaned if len(c.split()) >= 5]  # drop ultra-short
    if not cleaned:
        cleaned = [clean_caption(raw[-1])]
    best = max(cleaned, key=score_caption)
    return best, cleaned

best, alts = generate_best("golden hour by the sea", vibe="moody")
print("BEST:", best)
print("\nALTS:")
for a in alts[:5]: print("-", a)

BEST: Golden hour by the sea and with your #moodymenthats #colorcation

ALTS:
- Golden hour by the sea to favorite part weeks your conted belight b #moodyles
- Golden hour by the sea and with your #moodymenthats #colorcation


In [ ]:
import re

def clean_caption(text):

    text = re.sub(r"\s+", " ", text)

    text = re.sub(r"[^A-Za-z0-9# ,.!?]", "", text)

    text = re.sub(r"(#\w+)\w{3,}", r"\1", text)
    return text.strip()

In [ ]:
for vibe in ["moody","minimal","playful","wanderlust"]:
    best, _ = generate_best("golden hour by the sea", vibe=vibe)
    print(vibe, "→", best)

moody → golden hour by the sea #mood #intam #Colo #tourth #and
minimal → golden hour by the sea #minimalarit #GTAPELon #Loushic #war #sho
playful → golden hour by the sea #play #cupposiciala #byarch #encuse
wanderlust → golden hour by the sea #wanderlust #berchalfor #mighta #consurelfi #cu


In [ ]:
best = generate_best_caption("golden hour by the sea", vibe="moody", trials=60)
for i,s in enumerate(best,1):
    print(f"\n>>> {i}. {s}")



>>> 1. golden hour by the sea #moodyou ther to watets #andourn #hestonemarans and bexcke and and in with theatilleng

>>> 2. golden hour by the sea #moodyese this per ford and to the ever to @hielewartars ited whork by sery

>>> 3. golden hour by the sea #moodyss #consicae are and a for by shore the the sell of the taley trerict bencle com


In [ ]:
import re

def tidy_line(s, max_len=160):
    s = s.encode("ascii","ignore").decode("ascii")
    s = re.sub(r"[^A-Za-z0-9#@&.,!?'\"()\-/:; ]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    if len(s) > max_len:
        s = s[:max_len]
        s = re.sub(r"\s+\S*$", "", s).rstrip(" ,;:-")
    return s

In [ ]:
# quick polish finetune (optional)
polish_text = "\n".join(
  tidy_line(re.sub(r"[^\x20-\x7E]", " ", ln)) for ln in open("data/train.txt", encoding="utf-8")
)
open("data/train_ascii.txt", "w", encoding="utf-8").write(polish_text)
train_ids = encode(polish_text + "\n")

# 500 short steps, small LR
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, betas=(0.9,0.95), weight_decay=0.01)
for step in range(1, 501):
    ix = torch.randint(0, len(train_ids)-block_size-1, (batch_size,))
    xb = torch.stack([train_ids[i:i+block_size] for i in ix]).to(device)
    yb = torch.stack([train_ids[i+1:i+1+block_size] for i in ix]).to(device)
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True); loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0); optimizer.step()
    if step % 100 == 0:
        print(f"polish step {step} | loss {loss.item():.3f}")

polish step 100 | loss 2.762
polish step 200 | loss 2.580
polish step 300 | loss 2.520
polish step 400 | loss 2.422
polish step 500 | loss 2.339


In [ ]:
import torch.nn.functional as F

def generate_samples(prompt="", n=3, max_new_tokens=160, temperature=0.7, top_k=50):
    model.eval()
    out = []
    start = encode(prompt if prompt else "\n").unsqueeze(0).to(device)
    with torch.no_grad():
        for _ in range(n):
            idx = start.clone()
            for _ in range(max_new_tokens):
                idx_cond = idx[:, -block_size:]
                logits, _ = model(idx_cond)
                logits = logits[:, -1, :]
                logits = logits / max(temperature, 1e-8)

                if top_k is not None:
                    v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                    logits[logits < v[:, [-1]]] = -float('inf')

                probs = F.softmax(logits, dim=-1)
                next_id = torch.multinomial(probs, num_samples=1)
                idx = torch.cat([idx, next_id], dim=1)
                if int(next_id) == stoi.get("\n", -999):
                    break
            out.append(decode(idx[0].tolist()))
    model.train()
    return out

In [ ]:
import math, time, os

max_steps = 12000
log_every = 200
sample_every = 500
grad_clip = 1.0
best_val = float('inf')
ckpt_dir = "ckpts"; os.makedirs(ckpt_dir, exist_ok=True)
ckpt_path_best = os.path.join(ckpt_dir, "moodmuse_best.pt")

# simple cosine LR decay (keeps your current optimizer)
use_cosine_decay = True
base_lr = 3e-4
min_lr = 3e-5

def set_lr(step):
    if not use_cosine_decay:
        return
    t = step / max(1, max_steps)
    lr = min_lr + 0.5*(base_lr - min_lr)*(1 + math.cos(math.pi * t))
    for g in optimizer.param_groups:
        g['lr'] = lr

t0 = time.time()
for step in range(1, max_steps+1):
    set_lr(step)

    xb, yb = get_batch("train")
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
    optimizer.step()

    # quick val check
    if step % log_every == 0:
        with torch.no_grad():
            xb_val, yb_val = get_batch("val")
            _, vloss = model(xb_val, yb_val)
        elapsed = time.time() - t0
        print(f"step {step:5d} | train {loss.item():.3f} | val {vloss.item():.3f} | elapsed {elapsed/60:.1f} min")
        if vloss.item() < best_val:
            best_val = vloss.item()
            torch.save({"model": model.state_dict(),
                        "itos": itos, "stoi": stoi,
                        "block_size": block_size},
                       ckpt_path_best)
            print(f"  ↳ new best! saved to {ckpt_path_best}")

    if step % sample_every == 0:
        for s in generate_samples("", n=2, temperature=0.7, top_k=50):
            print("\n--- sample ---\n" + s)

step   200 | train 2.022 | val 2.060 | elapsed 0.3 min
  ↳ new best! saved to ckpts/moodmuse_best.pt
step   400 | train 1.760 | val 1.867 | elapsed 0.5 min
  ↳ new best! saved to ckpts/moodmuse_best.pt

--- sample ---

Chit sube today to estay this with a bag team into some boath its and the essent of the some in the turnals.


--- sample ---

📍 - - - - - - - - medischedds and can seen get the some and lighting as proletections weekend we had those for the teach to of the house, but in the cuts and th
step   600 | train 2.091 | val 2.186 | elapsed 0.8 min
step   800 | train 2.029 | val 1.987 | elapsed 1.1 min
step  1000 | train 1.921 | val 1.998 | elapsed 1.4 min

--- sample ---

Pass and pressent line in the Skistant Muris Crazara Geography pack to tracture left see the Document remally of our feel the come of the Chook More Happy Today

--- sample ---

The First Conton a time Program and start Sundaino house, the Pass Gally Contorica Day fining Maina a designs of of the look of the w

In [ ]:
ckpt = torch.load("ckpts/moodmuse_best.pt", map_location=device)
model.load_state_dict(ckpt["model"])
model.eval()

for s in generate_samples("", n=3, temperature=0.7, top_k=50):
    print("\n--- sample ---\n", s)


--- sample ---
 
Both @junellationess.h They are diving for the end of countryside of Barneta California and make up to my heart work in the world.


--- sample ---
 
When your submerged past 6 years old with your first special second and culture on your life now you asked around your favorite greats & to a time biggest way t

--- sample ---
 
1. @elisoperience



In [ ]:

import os
print("exists?", os.path.exists("ckpts/moodmuse_best.pt"))

exists? True


In [ ]:
# Saving the entire model to device
from google.colab import files
files.download("ckpts/moodmuse_best.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%%writefile inference.py
# inference.py — compat loader for trained checkpoint
import argparse
import json
import math
import os
from typing import List

import torch
import torch.nn as nn
import torch.nn.functional as F

# ---------- Utils ----------
def top_k_filter(logits, k=None):
    if k is None:
        return logits
    k = min(k, logits.size(-1))
    v, _ = torch.topk(logits, k)
    logits[logits < v[..., [-1]]] = -float("inf")
    return logits

# ---------- Compat Model ----------
class CompatSelfAttention(nn.Module):
    def __init__(self, n_embd, block_size, dropout=0.1):
        super().__init__()
        # fused qkv like in training code: weight shape [3*n_embd, n_embd]
        self.qkv = nn.Linear(n_embd, 3 * n_embd, bias=False)
        self.proj = nn.Linear(n_embd, n_embd, bias=False)
        self.attn_drop = nn.Dropout(dropout)
        self.proj_drop = nn.Dropout(dropout)
        # causal mask buffer (T, T) — 2D so it broadcasts cleanly to (B,T,T)
        mask = torch.tril(torch.ones(block_size, block_size, dtype=torch.bool))
        self.register_buffer("mask", mask, persistent=True)

    def forward(self, x):
        # Robustness: accept (B,1,T,C) by squeezing the singleton
        if x.dim() == 4 and x.size(1) == 1:
            x = x[:, 0]  # (B, T, C)
        if x.dim() != 3:
            raise ValueError(f"CompatSelfAttention expected (B,T,C); got {tuple(x.shape)}")

        B, T, C = x.shape
        qkv = self.qkv(x)                         # (B, T, 3C)
        q, k, v = qkv.chunk(3, dim=-1)            # each (B, T, C)

        # scaled dot-prod attention (single-head to match checkpoint)
        att = (q @ k.transpose(-2, -1)) / math.sqrt(C)      # (B, T, T)
        att = att.masked_fill(~self.mask[:T, :T], float("-inf"))
        att = F.softmax(att, dim=-1)
        att = self.attn_drop(att)
        y = att @ v                                         # (B, T, C)
        y = self.proj(y)
        y = self.proj_drop(y)
        return y

class CompatMLP(nn.Module):
    def __init__(self, n_embd, expansion=4, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, expansion * n_embd),
            nn.GELU(),
            nn.Linear(expansion * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    def forward(self, x): return self.net(x)

class CompatBlock(nn.Module):
    def __init__(self, n_embd, block_size, dropout=0.1):
        super().__init__()
        self.ln1 = nn.LayerNorm(n_embd)
        self.attn = CompatSelfAttention(n_embd, block_size, dropout=dropout)
        self.ln2 = nn.LayerNorm(n_embd)
        self.mlp = CompatMLP(n_embd, expansion=4, dropout=dropout)
    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

class CompatGPT(nn.Module):
    def __init__(self, vocab_size, n_embd, n_layer, block_size, dropout=0.1):
        super().__init__()
        self.block_size = block_size
        self.tok_emb = nn.Embedding(vocab_size, n_embd)
        # positional as (1, T, C) broadcast — implemented via nn.Embedding on index 0..T-1
        self.pos_emb = nn.Embedding(block_size, n_embd)
        self.blocks = nn.ModuleList([
            CompatBlock(n_embd, block_size, dropout=dropout) for _ in range(n_layer)
        ])
        self.ln_f = nn.LayerNorm(n_embd)
        # checkpoint has lm_head.weight only (no bias)
        self.lm_head = nn.Linear(n_embd, vocab_size, bias=False)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        if T > self.block_size:
            idx = idx[:, -self.block_size:]
            T = idx.size(1)
        pos = torch.arange(T, device=idx.device)               # (T,)
        x = self.tok_emb(idx) + self.pos_emb(pos)[None, :, :]  # (B, T, C)
        for blk in self.blocks:
            x = blk(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

# ---------- Loading checkpoint & tokenizer ----------
def load_checkpoint(ckpt_path, device):
    ckpt = torch.load(ckpt_path, map_location=device)

    # stoi/itos & block_size saved by training loop
    stoi = ckpt.get("stoi"); itos = ckpt.get("itos")
    block_size = ckpt.get("block_size", 128)

    # infer sizes from state_dict
    sd = ckpt["model"]
    vocab_size, n_embd = sd["tok_emb.weight"].shape
    n_layer = max(int(k.split(".")[1]) for k in sd.keys() if k.startswith("blocks.")) + 1

    model = CompatGPT(vocab_size=vocab_size, n_embd=n_embd, n_layer=n_layer, block_size=block_size)
    model.load_state_dict(sd, strict=True)
    model.to(device).eval()
    return model, stoi, itos, block_size

def encode(text, stoi):
    return torch.tensor([stoi[c] for c in text if c in stoi], dtype=torch.long)

def decode(ids, itos):
    return "".join(itos[int(i)] for i in ids)

# ---------- Generation ----------
@torch.no_grad()
def generate_caption(model, stoi, itos, block_size, desc="", vibe="aesthetic",
                     max_new_tokens=120, temperature=0.9, top_k=50, device="cpu"):
    # style nudger
    style_hint = f" #{vibe.replace(' ', '')}" if vibe else ""
    prompt = (desc.strip() + style_hint).strip() or "\n"

    idx = encode(prompt, stoi)[None, :].to(device)
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -block_size:]
        logits, _ = model(idx_cond)
        logits = logits[:, -1, :] / max(temperature, 1e-8)
        logits = top_k_filter(logits, top_k)
        probs = F.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, num_samples=1)
        idx = torch.cat([idx, next_id], dim=1)
        # stop early on newline for caption length vibe
        if int(next_id) == stoi.get("\n", -999):
            break
    return decode(idx[0], itos)

# ---------- CLI ----------
def main():
    p = argparse.ArgumentParser()
    p.add_argument("--ckpt", type=str, default="ckpts/moodmuse_best.pt")
    p.add_argument("--desc", type=str, default="golden hour on the beach")
    p.add_argument("--vibe", type=str, default="moody")
    p.add_argument("--max_new_tokens", type=int, default=120)
    p.add_argument("--temperature", type=float, default=0.9)
    p.add_argument("--top_k", type=int, default=60)
    args = p.parse_args()

    device = "cuda" if torch.cuda.is_available() else "cpu"
    if not os.path.exists(args.ckpt):
        raise FileNotFoundError(f"Checkpoint not found: {args.ckpt}")

    model, stoi, itos, block_size = load_checkpoint(args.ckpt, device)
    caption = generate_caption(
        model, stoi, itos, block_size,
        desc=args.desc, vibe=args.vibe,
        max_new_tokens=args.max_new_tokens,
        temperature=args.temperature,
        top_k=args.top_k,
        device=device
    )
    print("\n=== Caption ===")
    print(caption)

if __name__ == "__main__":
    main()


Writing inference.py


In [ ]:
!ls -lh

total 48K
drwxr-xr-x 2 root root 4.0K Sep  2 06:50 ckpts
drwxr-xr-x 2 root root 4.0K Sep  2 06:49 data
drwx------ 5 root root 4.0K Sep  2 07:04 drive
-rw-r--r-- 1 root root 8.3K Sep  2 07:04 inference.py
-rw-r--r-- 1 root root 7.9K Sep  2 06:29 inference_quick.py
-rw-r--r-- 1 root root   69 Sep  2 06:48 kaggle.json
drwxr-xr-x 5 root root 4.0K Sep  2 06:47 moodmuse
drwxr-xr-x 5 root root 4.0K Sep  2 06:47 MoodMuse
drwxr-xr-x 1 root root 4.0K Aug 28 13:43 sample_data


In [ ]:
%%writefile gpt.py
from dataclasses import dataclass
import math, torch
import torch.nn as nn
import torch.nn.functional as F

@dataclass
class GPTConfig:
    vocab_size: int
    block_size: int
    n_layer: int = 6
    n_head: int = 6
    n_embd: int = 384
    dropout: float = 0.1

class Head(nn.Module):
    def __init__(self, n_embd, head_size, block_size, dropout):
        super().__init__()
        self.key   = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # attention scores
        wei = q @ k.transpose(-2, -1) / math.sqrt(k.size(-1))  # (B,T,T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x) # (B,T,hs)
        out = wei @ v     # (B,T,hs)
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, n_embd, n_head, block_size, dropout):
        super().__init__()
        assert n_embd % n_head == 0
        head_size = n_embd // n_head
        self.heads = nn.ModuleList([Head(n_embd, head_size, block_size, dropout) for _ in range(n_head)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head, block_size, dropout):
        super().__init__()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        self.sa  = MultiHeadAttention(n_embd, n_head, block_size, dropout)
        self.ffwd = FeedForward(n_embd, dropout)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPT(nn.Module):
    def __init__(self, config: GPTConfig):
        super().__init__()
        self.config = config
        self.token_embedding_table = nn.Embedding(config.vocab_size, config.n_embd)
        self.position_embedding_table = nn.Embedding(config.block_size, config.n_embd)
        self.blocks = nn.ModuleList([Block(config.n_embd, config.n_head, config.block_size, config.dropout) for _ in range(config.n_layer)])
        self.ln_f = nn.LayerNorm(config.n_embd)
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)                      # (B,T,C)
        pos = torch.arange(0, T, device=idx.device, dtype=torch.long)
        pos_emb = self.position_embedding_table(pos)[None, :, :]       # (1,T,C)
        x = tok_emb + pos_emb
        for block in self.blocks:
            x = block(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)                                       # (B,T,vocab)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None, stop_id=None):
        self.eval()
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.config.block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / max(temperature, 1e-8)
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('inf')
            probs = F.softmax(logits, dim=-1)
            next_id = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, next_id], dim=1)
            if stop_id is not None and int(next_id) == int(stop_id):
                break
        return idx


Writing gpt.py


In [ ]:
!ls ckpts

moodmuse_best.pt


In [ ]:
!python inference.py --ckpt ckpts/moodmuse_best.pt --desc "sunset over the city skyline" --vibe moody

Traceback (most recent call last):
  File "/content/inference.py", line 182, in <module>
    main()
  File "/content/inference.py", line 169, in main
    model, stoi, itos, block_size = load_checkpoint(args.ckpt, device)
                                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/inference.py", line 122, in load_checkpoint
    model.load_state_dict(sd, strict=True)
  File "/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py", line 2624, in load_state_dict
    raise RuntimeError(
RuntimeError: Error(s) in loading state_dict for CompatGPT:
	size mismatch for blocks.0.attn.mask: copying a param with shape torch.Size([1, 1, 128, 128]) from checkpoint, the shape in current model is torch.Size([128, 128]).
	size mismatch for blocks.1.attn.mask: copying a param with shape torch.Size([1, 1, 128, 128]) from checkpoint, the shape in current model is torch.Size([128, 128]).
	size mismatch for blocks.2.attn.mask: copying a param with shape torch.Size([1, 

In [ ]:
!python inference.py --ckpt ckpts/moodmuse_best.pt \
  --desc "sunset over the city skyline" \
  --vibe "minimal chic"

Traceback (most recent call last):
  File "/content/inference.py", line 182, in <module>
    main()
  File "/content/inference.py", line 169, in main
    model, stoi, itos, block_size = load_checkpoint(args.ckpt, device)
                                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/inference.py", line 122, in load_checkpoint
    model.load_state_dict(sd, strict=True)
  File "/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py", line 2624, in load_state_dict
    raise RuntimeError(
RuntimeError: Error(s) in loading state_dict for CompatGPT:
	size mismatch for blocks.0.attn.mask: copying a param with shape torch.Size([1, 1, 128, 128]) from checkpoint, the shape in current model is torch.Size([128, 128]).
	size mismatch for blocks.1.attn.mask: copying a param with shape torch.Size([1, 1, 128, 128]) from checkpoint, the shape in current model is torch.Size([128, 128]).
	size mismatch for blocks.2.attn.mask: copying a param with shape torch.Size([1, 

In [ ]:
# a bit cooler + narrower sampling
!python inference.py --ckpt ckpts/moodmuse_best.pt \
  --desc "sunset over the city skyline" --vibe "moody" \
  --temperature 0.8 --top_k 40 --max_new_tokens 60

# warmer, broader, 3 candidates (just run 3 times for now)
!python inference.py --ckpt ckpts/moodmuse_best.pt \
  --desc "sunset over the city skyline" --vibe "minimal chic" \
  --temperature 0.95 --top_k 80 --max_new_tokens 80


Traceback (most recent call last):
  File "/content/inference.py", line 182, in <module>
    main()
  File "/content/inference.py", line 169, in main
    model, stoi, itos, block_size = load_checkpoint(args.ckpt, device)
                                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/inference.py", line 122, in load_checkpoint
    model.load_state_dict(sd, strict=True)
  File "/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py", line 2624, in load_state_dict
    raise RuntimeError(
RuntimeError: Error(s) in loading state_dict for CompatGPT:
	size mismatch for blocks.0.attn.mask: copying a param with shape torch.Size([1, 1, 128, 128]) from checkpoint, the shape in current model is torch.Size([128, 128]).
	size mismatch for blocks.1.attn.mask: copying a param with shape torch.Size([1, 1, 128, 128]) from checkpoint, the shape in current model is torch.Size([128, 128]).
	size mismatch for blocks.2.attn.mask: copying a param with shape torch.Size([1, 

In [ ]:
!python inference.py --ckpt ckpts/moodmuse_best.pt --desc "golden hour on the beach" --vibe "soft grunge"

Traceback (most recent call last):
  File "/content/inference.py", line 182, in <module>
    main()
  File "/content/inference.py", line 169, in main
    model, stoi, itos, block_size = load_checkpoint(args.ckpt, device)
                                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/inference.py", line 122, in load_checkpoint
    model.load_state_dict(sd, strict=True)
  File "/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py", line 2624, in load_state_dict
    raise RuntimeError(
RuntimeError: Error(s) in loading state_dict for CompatGPT:
	size mismatch for blocks.0.attn.mask: copying a param with shape torch.Size([1, 1, 128, 128]) from checkpoint, the shape in current model is torch.Size([128, 128]).
	size mismatch for blocks.1.attn.mask: copying a param with shape torch.Size([1, 1, 128, 128]) from checkpoint, the shape in current model is torch.Size([128, 128]).
	size mismatch for blocks.2.attn.mask: copying a param with shape torch.Size([1, 

In [ ]:
# 1) making folders
!mkdir -p MoodMuse/{src,data/raw,data/processed,outputs/checkpoints,outputs/samples}

# 2) keeping empty dirs tracked by git later
!touch MoodMuse/data/.gitkeep
!touch MoodMuse/outputs/.gitkeep

# 3) moving into the project folder for the rest of the session
!cd MoodMuse && pwd && ls -la

/content/MoodMuse
total 20
drwxr-xr-x 5 root root 4096 Sep  2 16:00 .
drwxr-xr-x 1 root root 4096 Sep  2 16:29 ..
drwxr-xr-x 4 root root 4096 Sep  2 16:33 data
drwxr-xr-x 4 root root 4096 Sep  2 16:33 outputs
drwxr-xr-x 2 root root 4096 Sep  2 16:00 src


In [ ]:
import torch, json
import torch.nn.functional as F
from pathlib import Path

device = "cuda" if torch.cuda.is_available() else "cpu"

# --- load tokenizer ---
tok = json.loads(Path("data/tokenizer_char.json").read_text(encoding="utf-8"))
stoi = {k: int(v) for k, v in tok["stoi"].items()}
itos = {int(k): v for k, v in tok["itos"].items()}
def encode(s): return torch.tensor([stoi[c] for c in s if c in stoi], dtype=torch.long)
def decode(ids): return "".join(itos[int(i)] for i in ids)

# --- load model (same class/arch trained) ---
from gpt import GPTConfig, GPT
vocab_size = len(itos)
block_size = 128
cfg = GPTConfig(vocab_size=vocab_size, block_size=block_size,
                n_layer=4, n_head=4, n_embd=384, dropout=0.2)
model = GPT(cfg).to(device)
ckpt = torch.load("/content/drive/MyDrive/moodmuse_ckpts/moodmuse_best.pt", map_location=device)
model.load_state_dict(ckpt["model"])
model.eval()

@torch.no_grad()
def generate_caption(description:str,
                     vibe:str="aesthetic",
                     max_new_tokens:int=160,
                     temperature:float=0.8,
                     top_p:float=0.9):
    """
    description: your photo blurb (e.g., 'golden-hour beach, footprints in sand')
    vibe: short style tag ('minimal', 'wholesome', 'moody', 'luxury', 'playful', etc.)
    """
    # a tiny prompt that nudges IG-style
    prompt = f"{vibe.title()} vibe — {description}\n"
    idx = encode(prompt).unsqueeze(0).to(device)

    for _ in range(max_new_tokens):
        idx_cond = idx[:, -block_size:]
        logits, _ = model(idx_cond)
        logits = logits[:, -1, :] / max(temperature, 1e-8)
        # nucleus sampling
        probs = F.softmax(logits, dim=-1)
        sorted_probs, sorted_idx = torch.sort(probs, descending=True)
        cumsum = torch.cumsum(sorted_probs, dim=-1)
        mask = cumsum > top_p
        mask[..., 0] = False
        sorted_probs[mask] = 0
        sorted_probs = sorted_probs / (sorted_probs.sum(dim=-1, keepdim=True) + 1e-8)
        next_id = sorted_idx.gather(-1, torch.multinomial(sorted_probs, num_samples=1))
        idx = torch.cat([idx, next_id], dim=1)
        if int(next_id) == stoi.get("\n", -999):  # stop at newline
            break

    raw = decode(idx[0].tolist())
    generated = raw[len(prompt):].strip()

    # simple post-processing
    generated = generated.replace("..", ".").replace("  ", " ").strip()
    # cap length to ~2200 chars
    if len(generated) > 350:
        generated = generated[:350].rsplit(" ", 1)[0] + "…"
    return generated

# quick sanity:
print(generate_caption("golden-hour coffee on a window sill", vibe="minimal"))
print(generate_caption("rainy neon street reflection", vibe="moody"))

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/moodmuse_ckpts/moodmuse_best.pt'

In [ ]:
# === Demo-polish helpers (drop this in a new cell) ===
import re, random
import torch
import torch.nn.functional as F

# 1) Decoding tweaks
DECODE_T = 0.7        # lower temp -> fewer weird neologisms
DECODE_TOPK = 40      # tighter top-k
N_SAMPLES = 8         # generate multiple and pick the cleanest

# 2) A small, safe hashtag whitelist you’re happy to show
HASHTAG_WHITELIST = [
    "moody","minimal","playful","wanderlust","sunset","vibes","aesthetic",
    "goldenhour","seaside","cityscape","film","portrait","travel","weekend",
    "softlight","summer","cozy","streetstyle"
]

# 3) helpers
def norm_token(t): return re.sub(r"[^A-Za-z0-9#]", "", t)

def pick_whitelist_tag(raw_tag):
    """Map a fuzzy model tag to your whitelist, else return ''."""
    t = raw_tag.lower().lstrip("#")
    if not t: return ""
    # exact
    if t in HASHTAG_WHITELIST: return f"#{t}"
    # simple similarity on prefix/containment
    best = ""
    best_score = 0.0
    for w in HASHTAG_WHITELIST:
        # overlap score: common prefix length / max len
        pref = 0
        for a,b in zip(t, w):
            if a==b: pref += 1
            else: break
        score = max(pref/len(w), pref/max(1,len(t)))
        if t in w or w in t: score = max(score, 0.66)
        if score > best_score:
            best_score, best = score, w
    return f"#{best}" if best_score >= 0.5 else ""  # require a decent match

def clean_caption(text, vibe=None, keep_tags=2):
    # keep ASCII, collapse space
    text = re.sub(r"[^\x20-\x7E]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()

    # split words & tags
    toks = [norm_token(t) for t in text.split()]
    words, tags = [], []
    for t in toks:
        if t.startswith("#"):
            m = pick_whitelist_tag(t)
            if m and m not in tags:
                tags.append(m)
        else:
            # filter obvious garbage: mega-long or mostly consonants
            if len(t) <= 20 and not re.search(r"[bcdfghjklmnpqrstvwxyz]{5,}", t.lower()):
                words.append(t)

    # ensure vibe tag present (mapped to whitelist if possible)
    if vibe:
        vtag = pick_whitelist_tag("#" + vibe)
        if vtag and vtag not in tags:
            tags = [vtag] + tags

    # keep up to N tags, unique
    tags = tags[:keep_tags]

    # sentencecase words and limit to 8–14 tokens for IG feel
    if not words:
        words = ["Golden", "hour", "by", "the", "sea"]
    words = words[:14]
    sent = " ".join(words).strip()
    if sent:
        sent = sent[0].upper() + sent[1:]
        if not re.search(r"[.!?]$", sent):
            sent += "."
    return (sent + (" " + " ".join(tags) if tags else "")).strip()

@torch.no_grad()
def sample_once(model, stoi, itos, block_size, prompt, device="cuda", T=DECODE_T, top_k=DECODE_TOPK, max_new_tokens=64):
    # encode
    ids = torch.tensor([stoi.get(c, None) for c in prompt if c in stoi], dtype=torch.long)
    if ids.numel() == 0:
        ids = torch.tensor([stoi.get("\n")], dtype=torch.long)
    ids = ids.unsqueeze(0).to(device)

    for _ in range(max_new_tokens):
        idx_cond = ids[:, -block_size:]
        logits, _ = model(idx_cond)
        logits = logits[:, -1, :] / max(T, 1e-8)
        if top_k is not None:
            v,_ = torch.topk(logits, min(top_k, logits.size(-1)))
            logits[logits < v[:,[-1]]] = -float("inf")
        probs = F.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, num_samples=1)
        ids = torch.cat([ids, next_id], dim=1)
        # optional early stop on newline
        if int(next_id) == stoi.get("\n", -999):
            break
    return "".join(itos[int(i)] for i in ids[0])

def generate_best(desc, vibe="moody", tries=N_SAMPLES):
    # style hint appended to steer tone
    base = (desc.strip() + f" #{vibe}".replace(" ", "")).strip()
    cands = []
    for _ in range(tries):
        raw = sample_once(model, stoi, itos, block_size, prompt=base, device=device)
        cleaned = clean_caption(raw, vibe=vibe, keep_tags=2)
        # score: shorter & more whitelisted tags preferred
        tag_count = len(re.findall(r"#\w+", cleaned))
        nonsense_pen = 1 if re.search(r"[bcdfghjklmnpqrstvwxyz]{6,}", cleaned.lower()) else 0
        score = -len(cleaned) - 10*nonsense_pen + 8*tag_count
        cands.append((score, cleaned))
    cands.sort(reverse=True, key=lambda x: x[0])
    return cands[0][1], cands   # best, all

# === Run demo for multiple vibes ===
test_desc = "golden hour by the sea"
for v in ["moody","minimal","playful","wanderlust"]:
    best, _ = generate_best(test_desc, vibe=v)
    print(f"{v:10s} → {best}")

moody      → Golden hour by the sea#moody. #moody
minimal    → Golden hour by the sea#minimally. #minimal
playful    → Golden hour by the sea#playful. #playful
wanderlust → Golden hour by the sea#wanderlust. #wanderlust


In [ ]:
# --- PATCH: better cleaner + smart second tag ---

# simple keyword→tag map for a tasteful second hashtag
KEYWORD_TAGS = {
    "golden": "goldenhour",
    "sunset": "sunset",
    "dawn": "goldenhour",
    "dusk": "goldenhour",
    "beach": "seaside",
    "sea": "seaside",
    "ocean": "seaside",
    "city": "cityscape",
    "street": "streetstyle",
    "film": "film",
    "travel": "travel",
    "summer": "summer",
    "cozy": "cozy",
}

def second_tag_from_desc(desc: str):
    d = desc.lower()
    for k, tag in KEYWORD_TAGS.items():
        if k in d:
            return f"#{tag}"
    return ""


def clean_caption(text, vibe=None, keep_tags=2, desc_for_extra_tag=""):
    # make sure hashtags have a leading space if glued to a word
    text = re.sub(r"([A-Za-z0-9])(#)", r"\1 \2", text)

    # ASCII only + collapse spaces
    text = re.sub(r"[^\x20-\x7E]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()

    # tokenize and normalize
    toks = [norm_token(t) for t in text.split()]
    words, tags = [], []
    for t in toks:
        if t.startswith("#"):
            m = pick_whitelist_tag(t)
            if m and m not in tags:
                tags.append(m)
        else:
            # filter obvious junk
            if 1 <= len(t) <= 20 and not re.search(r"[bcdfghjklmnpqrstvwxyz]{5,}", t.lower()):
                words.append(t)

    # ensure vibe tag (once)
    if vibe:
        vtag = pick_whitelist_tag("#" + vibe)
        if vtag:
            tags = [t for t in tags if t.lower() != vtag.lower()]  # de-dup any fuzzy version
            tags.insert(0, vtag)

    # optional second tag from description, if we still have room
    if len(tags) < keep_tags and desc_for_extra_tag:
        extra = second_tag_from_desc(desc_for_extra_tag)
        extra = pick_whitelist_tag(extra) if extra else ""
        if extra and extra not in tags:
            tags.append(extra)

    # cap tag count
    tags = tags[:keep_tags]

    # sentencecase, short & tidy
    if not words:
        words = ["Golden", "hour", "by", "the", "sea"]
    words = words[:14]
    sent = " ".join(words).strip()
    # remove any trailing period directly before a hashtag (we’ll add one properly)
    sent = re.sub(r"\.(\s*#)", r"\1", sent)
    if sent:
        sent = sent[0].upper() + sent[1:]
        if not re.search(r"[.!?]$", sent):
            sent += "."

    return (sent + (" " + " ".join(tags) if tags else "")).strip()


def generate_best(desc, vibe="moody", tries=N_SAMPLES):
    base = (desc.strip() + f" #{vibe}".replace(" ", "")).strip()
    cands = []
    for _ in range(tries):
        raw = sample_once(model, stoi, itos, block_size, prompt=base, device=device)
        cleaned = clean_caption(raw, vibe=vibe, keep_tags=2, desc_for_extra_tag=desc)
        # score: shorter, with 2 tags gets a small bonus
        tag_count = len(re.findall(r"#\w+", cleaned))
        nonsense_pen = 1 if re.search(r"[bcdfghjklmnpqrstvwxyz]{6,}", cleaned.lower()) else 0
        score = -len(cleaned) - 10*nonsense_pen + 6*tag_count
        cands.append((score, cleaned))
    cands.sort(reverse=True, key=lambda x: x[0])
    return cands[0][1], cands

In [ ]:
for v in ["moody","minimal","playful","wanderlust"]:
    print(f"{v:10s} → {generate_best('golden hour by the sea', vibe=v)[0]}")


moody      → Golden hour by the sea. #moody #goldenhour
minimal    → Golden hour by the sea. #minimal #travel
playful    → Golden hour by the sea. #playful #sunset
wanderlust → Golden hour by the sea. #wanderlust #sunset


In [ ]:
%%bash
# (A) create folders
cd /content
mkdir -p MoodMuse/{src,data,outputs/ckpts,outputs/samples}

# (B) copying working assets into the repo
#  - the tokenizer built in the notebook
#  - the best checkpoint trained (the one with tok_emb.* keys)
cp -v /content/data/tokenizer_char.json /content/MoodMuse/data/
cp -v /content/ckpts/moodmuse_best.pt  /content/MoodMuse/outputs/ckpts/


'/content/data/tokenizer_char.json' -> '/content/MoodMuse/data/tokenizer_char.json'
'/content/ckpts/moodmuse_best.pt' -> '/content/MoodMuse/outputs/ckpts/moodmuse_best.pt'


In [ ]:
%%bash
cp -v /content/inference_quick.py /content/MoodMuse/src/inference.py

cp: cannot stat '/content/inference_quick.py': No such file or directory


CalledProcessError: Command 'b'# if your working script is /content/inference_quick.py\ncp -v /content/inference_quick.py /content/MoodMuse/src/inference.py\n'' returned non-zero exit status 1.